<a href="https://colab.research.google.com/github/Jin-jin-jara/Deep-Learning/blob/master/CNN_source.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# 파일이 있으면 지우고 새로운 파일 만들기
!rm -rf imagenet
!mkdir imagenet

!rm -rf dogs.tar.gz
!ls -al  

# 압축 풀기
!tar xvfz dogs.tar.gz
!ls -al dogs

In [0]:
# 데이터 다운 받아 가져오기
# 버섯
!wget -O imagenet/mushroom1.jpg http://farm4.static.flickr.com/3023/2822584107_186167ff68.jpg
!wget -O imagenet/mushroom2.jpg http://farm3.static.flickr.com/2416/1593642808_efcef6c9c2.jpg
!wget -O imagenet/mushroom3.jpg http://farm4.static.flickr.com/3003/2536991564_5f9b2f5b53.jpg
  
# 강아지
!wget -O imagenet/dog1.jpg http://farm1.static.flickr.com/58/160964915_d708f48d0d.jpg
!wget -O imagenet/dog2.jpg http://farm1.static.flickr.com/51/144906086_049df05364.jpg
!wget -O imagenet/dog3.jpg http://farm3.static.flickr.com/2133/2236535445_ca650757f2.jpg
  
# 고양이  
!wget -O imagenet/cat1.jpg http://farm1.static.flickr.com/131/393656824_bd89c512d0.jpg
!wget -O imagenet/cat2.jpg http://farm1.static.flickr.com/213/505539125_d7193beb76.jpg
!wget -O imagenet/cat3.jpg http://farm1.static.flickr.com/24/63785988_c16c10b4e5.jpg

!wget https://github.com/dhrim/bmac_seminar/raw/master/material/dogs.tar.gz
  

In [0]:
# CNN을 위한 라이브러리 가져오기
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications import vgg16
import numpy as np
import matplotlib.pyplot as plt

In [0]:
# 이미지 데이터 로드하고 넘파이로 변환하고 등등 케라스에서도 제공하지만 수동으로 해야할 경우
def load_file_names_and_category_names(path):
  
  file_names = []
  category_names = []

  dir_names = os.listdir(path)
      
  for dir_name in dir_names:
    file_names_in_a_dir = os.listdir(path+"/"+dir_name)

    for a_file_name in file_names_in_a_dir:
      full_file_name = path+"/"+dir_name+"/"+a_file_name
      file_names.append(full_file_name)
      category_names.append(dir_name)

  return file_names, category_names


def load_image_files_into_numpy_array(file_names):
  file_count = len(file_names)    
  data = np.ndarray(shape=(file_count,224,224,3), dtype=np.float64)    
  for i in range(len(file_names)):
    image = load_img(file_names[i], target_size=(224, 224))
    image = img_to_array(image)
    data[i] = image
  return data


def load_custom_data(path, train_ratio=0.8):
  
  file_names, category_names = load_file_names_and_category_names(path)
  x = load_image_files_into_numpy_array(file_names)
  
  labels, y = np.unique(category_names, return_inverse=True)  

  s = int(x.shape[0]*train_ratio)
  train_x, test_x = x[:s], x[s:]
  train_y, test_y = y[:s], y[s:]
  
  return (train_x, train_y), (test_x, test_y), labels



In [0]:
(train_x, train_y), (test_x, test_y), custom_labels = load_custom_data("dogs")

train_x = vgg16.preprocess_input(train_x)
test_x = vgg16.preprocess_input(test_x)

print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)
print(custom_labels)  


In [0]:
print(train_x[0])
print(train_y[0])

In [0]:
# 기존에 학습된 모델 파일 삭제
vgg16_model_path = 'new_trained_from_vgg16.h5'
!rm -rf {vgg16_model_path}

In [0]:
import os

import matplotlib.pyplot as plt
import numpy as np

from tensorflow.keras import optimizers
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array

# from tensorflow.keras.applications import vgg16
# from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import resnet50   # 모듈이름
from tensorflow.keras.applications import ResNet50   # 클래스 이름


# conv_layers = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
conv_layers = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3)) # 뒤에 densenet부분이 top 즉 include_top=False라면 feature extraction부분만 쓰겠다는 의미
# Convolution Layer를 학습되지 않도록 고정 (실제 vgg와 테스크가 다르니까 다르게 써야하는데, 이미 학습이 잘 된 모델을 가져왔으니까 convolution쪽은 학습하지마 그대로 쓸거니까)
conv_layers.trainable = False

# 새로운 모델 생성하기
model = models.Sequential()

# VGG16모델의 Convolution Layer를 추가
# 여긴 trainable이 False니까 학습이 안되고
model.add(conv_layers)

# 여기가 학습되는 부분
# 모델의 Fully Connected 부분을 재구성
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(10, activation='softmax'))

# 모델
model.compile(loss='sparse_categorical_crossentropy', optimizer="RMSprop", metrics=['acc'])


data_aug_generator = ImageDataGenerator(   # train 데이터 증강 제너레이터
      rotation_range=10,
      width_shift_range=0.1,
      height_shift_range=0.1,
      horizontal_flip=True,
      vertical_flip=False,
#       preprocessing_function=vgg16.preprocess_input
      preprocessing_function=resnet50.preprocess_input # 여기에 preprocess 함수를 설정
)


data_no_aug_generator = ImageDataGenerator(   # test는 증강시킬 필요 없으니까 no_aug
#       preprocessing_function=vgg16.preprocess_input
      preprocessing_function=resnet50.preprocess_input
)

# flow_from_directory 메소드를 활용하면 폴더 형태로된 데이터 구조를 바로 가져와서 사용할 수 있다.
train_data_generator = data_aug_generator.flow_from_directory(  # 이 디렉토리에 있는 데이터를 증강시켜라
      "dogs_prepared/train",
      target_size=(224,224),
      batch_size=64,
      class_mode='sparse'
)

test_data_generator = data_no_aug_generator.flow_from_directory(  # 증강시킬 필요 없고 
      "dogs_prepared/test",
      target_size=(224,224),
      class_mode='sparse'
)

# 여기선 fit이 아니라 fit_generator를 쓴다
model.fit_generator(
      train_data_generator,
      validation_data=test_data_generator,
      validation_steps=5,       # 매 에포크마다 벨리데이션
      steps_per_epoch=train_data_generator.samples/64,  # data_size / batch_size
      epochs=10
)

# model.fit(train_x, train_y, epochs=100, batch_size=64, shuffle=True, validation_split=0.1)

# 모델 저장하기
model.save(vgg16_model_path)

y_ = model.predict_generator(
      test_data_generator,
      steps=test_data_generator.samples/64
)


In [0]:
# 결과 확인 
import matplotlib.pyplot as plt

loss, acc = model.evaluate(test_x, test_y)
print("loss =",loss)
print("acc =",acc)

custom_labels = list(test_data_generator.class_indices.keys())
predicted = np.argmax(y_, axis=1)
print(predicted[0], custom_labels[predicted[0]])

# train_x, test_x는 vgg16.prerprocess_input()에 의해 변형되었다.
(_, _), (raw_test_x, _), custom_labels = load_custom_data("dogs")


for i in [0,50,100,150,200]:  
  print(test_y[i], custom_labels[test_y[i]])
  print(predicted[i], custom_labels[predicted[i]])
  plt.imshow(raw_test_x[i].astype(int))
  plt.show()